### Notebook: Focus on chunking data

In [11]:
!pip install --quiet langchain_experimental langchain_openai 
!pip install -qU "langchain-chroma>=0.1.2"


zsh:1: 0.1.2 not found


In [2]:
import pandas as pd

ucsc_passages_df = pd.read_csv("../passage.csv", index_col=0)
ucsc_passages_df

,passage,link,comments
Index,,,
0,"Our club exists to give community members, UC ...",https://recreation.ucsc.edu/boating/community-...,NaN
1,Boating Club Hours\nThe Community Boating Cent...,https://recreation.ucsc.edu/boating/index.html,NaN
2,Our boats:\nSailing vessels for weekend use cu...,https://recreation.ucsc.edu/boating/community-...,NaN
3,Club Membership Fees:\n\nUCSC Student Membersh...,https://recreation.ucsc.edu/boating/community-...,NaN
4,Summer Boating Classes\nEnrollment for summer ...,https://recreation.ucsc.edu/boating/index.html,NaN
...,...,...,...
59,Transfer Admission Guarantee\nTransfer Admissi...,https://admissions.ucsc.edu/transfer-admission...,NaN
60,UC Santa Cruz\nWith its jaw-dropping coastline...,https://admission.universityofcalifornia.edu/c...,NaN
61,Santa Cruz: First-year admit data\nThis is a s...,https://admission.universityofcalifornia.edu/c...,NaN


In [4]:
from langchain_community.document_loaders import DataFrameLoader

ucsc_passge_data_loader = DataFrameLoader(ucsc_passages_df, page_content_column="passage")
documents = ucsc_passge_data_loader.load()


In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
embed_model = OpenAIEmbeddings()


semantic_splitter = SemanticChunker(embed_model, breakpoint_threshold_type="percentile")

In [6]:
semantic_chunks = semantic_splitter.create_documents([d.page_content for d in documents])


In [7]:


print(semantic_chunks[0].page_content)
print(len(semantic_chunks[0].page_content))

Our club exists to give community members, UC Santa Cruz students, faculty, and staff access to equipment that makes it possible to enjoy our beautiful Monterey Bay. The club provides members a unique opportunity to use any number of sailing and rowing vessels under the weekend supervision of the Boating Center dockmaster.
324


In [8]:
from langchain_chroma import Chroma

persist_directory = './data'

vectorstore = Chroma.from_documents(documents=semantic_chunks, embedding=embed_model, persist_directory=persist_directory)

### Testing the retriever

In [9]:
vectorstore.similarity_search("how much does summer school cost at ucsc", k=2)

[Document(page_content='Undergraduate Students\nUCSC students singing at an outdoor concert \nStandard budgets are used to determine how much financial aid students receive. UC establishes standard budgets for students using statistical data from a variety of sources, including periodic surveys of UC Santa Cruz students. Students are assigned one of the 3 budget types- On-Campus, Off-Campus, or Commuter- based on the housing plan they selected when completing the FAFSA or Dream Application. Standard budgets do not include living expenses for winter and spring breaks. Click on the links below to explore the Undergraduate Budgets for entering and continuing students. For 2023-24 Budgets, click here. 2024-25 Undergraduate Budgets for Entering Students - Fall, Winter, & Spring Terms [hide] \nEntering Students\t\nOn-Campus\n\nOff-Campus\n\nCommuter\n\nTuition and Fees   **\t\n$16,014\n\n$16,014\n\n$16,014\n\nLiving Expenses  *\t\n$19,950\n\n$19,560\n\n$8,022\n\nBooks, Course Materials, Supp